In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
from fastai.vision import *
import pandas as pd

In [6]:
df=pd.read_csv('./ori.csv')

In [7]:
df

,patientid,healthassessmentid,assessment_date,n_assessment,assessment_number,final_assessment,assessment_year,assessment_type,assessment_quarter,assessment_month,...,imd_idaopi_score,glucose,tot_cholesterol,alcohol_consumption,regular_exercise,t2dm_nice,prediabetic,first_to_last,days_between_ha,years_between_ha
0,0x0001EC929E1AB41684A7B5528628E4B7,0x97B5EB4313702E21828DD59E5BC4F085,13oct2017,1,1,1,2017,1,4,10,...,0.024,3.8,4.68,1,0,0,0,NaN,NaN,NaN
1,0x0004022CC26B79C37471E953F521D0DC,0x786B53CCD0FF6397100F4080465628E0,01may2018,1,1,1,2018,4,2,5,...,0.110,5.3,6.20,1,1,0,0,NaN,NaN,NaN
2,0x00044057F135CC6526BE752AD83115F6,0x6B04800E0E852F8C997D9AD8019B868E,13apr2018,1,1,1,2018,3,2,4,...,0.120,5.6,6.10,2,1,0,1,NaN,NaN,NaN
3,0x000527A13F12637CFD34616163AFBC48,0xB0E4800F18AD1367A01658AA1A7736EF,12may2014,2,1,0,2014,4,2,5,...,0.071,5.9,4.60,0,1,0,1,NaN,NaN,NaN
4,0x000527A13F12637CFD34616163AFBC48,0x6A1A149F1B99B942F3C3938A85799151,21mar2016,2,2,1,2016,4,1,3,...,0.071,5.3,4.40,0,1,0,0,2.0,679.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117214,0xFFFE9F664C2DDBA4A37BCD35936C7422,0x992C4F48033E827CF469A091336D52A0,20jul2016,2,1,0,2016,3,3,7,...,0.051,5.5,3.50,1,1,0,0,NaN,NaN,NaN
117215,0xFFFE9F664C2DDBA4A37BCD35936C7422,0xA000B761970ED9C98BA3729B153FD289,18jan2018,2,2,1,2018,3,1,1,...,0.051,5.0,3.90,1,1,0,0,1.0,547.0,1.0
117216,0xFFFEF21D8708966769854E3B2EE1D475,0x2060701EE25C96F092C19996863B09CC,09jun2017,1,1,1,2017,4,2,6,...,0.047,5.2,5.50,1,1,0,0,NaN,NaN,NaN
117217,0xFFFF41A5B6018F5D524A5CA496301210,0xAF2586E8B021F7A71CAB6AB101C17056,26jul2016,1,1,1,2016,4,3,7,...,0.064,5.1,4.10,1,1,0,0,NaN,NaN,NaN


In [8]:
# Filter patients who have multiple assessments
multiple_assessments = df.groupby('patientid').filter(lambda x: len(x) > 1)

# Create a function to check if a patient went from not prediabetic to prediabetic
def check_transition(group):
    # Sort by assessment date
    group = group.sort_values('assessment_date')
    # Check if first assessment was not prediabetic and last assessment was prediabetic
    return (group['prediabetic'].iloc[0] == 0) and (group['prediabetic'].iloc[-1] == 1)

# Apply the function to each patient group
prediabetic_transitions = multiple_assessments.groupby('patientid').apply(check_transition)

# Filter patients who transitioned
transitioned_patients = prediabetic_transitions[prediabetic_transitions].index

# Get the data for these patients
prediabetes_progression = df[df['patientid'].isin(transitioned_patients)].sort_values(['patientid', 'assessment_date'])

# Display the count and patient data
print(f"Number of patients who went from not having prediabetes to having prediabetes: {len(transitioned_patients)}")
prediabetes_progression

Number of patients who went from not having prediabetes to having prediabetes: 2565


,patientid,healthassessmentid,assessment_date,n_assessment,assessment_number,final_assessment,assessment_year,assessment_type,assessment_quarter,assessment_month,...,imd_idaopi_score,glucose,tot_cholesterol,alcohol_consumption,regular_exercise,t2dm_nice,prediabetic,first_to_last,days_between_ha,years_between_ha
34,0x0014CB06991F127CC91B006823AD9726,0x3640DD43418647144D78EFD4B9BFD659,10dec2014,2,1,0,2014,4,4,12,...,0.191,5.2,4.00,2,1,0,0,NaN,NaN,NaN
35,0x0014CB06991F127CC91B006823AD9726,0x39CA627C758A0EED8BE9717D0C88A9B3,19dec2016,2,2,1,2016,3,4,12,...,0.191,5.5,4.20,1,0,0,1,2.0,740.0,2.0
165,0x0066C9097D9BD3BD0EE283DD087E23B5,0x225D0F0E973D16355A50816445804410,07dec2016,2,1,0,2016,3,4,12,...,0.066,5.3,6.70,2,1,0,0,NaN,NaN,NaN
166,0x0066C9097D9BD3BD0EE283DD087E23B5,0xF29BFC22BC35784D66EB449E3F24B21B,12mar2018,2,2,1,2018,3,1,3,...,0.066,5.6,6.90,2,1,0,1,1.0,460.0,1.0
361,0x00D84A27CDC52F77B5010272BA916675,0xD58FCCC4F256FCC1EBE305D9A47BD54B,24mar2016,2,1,0,2016,1,1,3,...,0.072,4.9,5.19,1,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117080,0xFF9D3A3258DFB85307D3110C534C9B57,0x91D0B897155A4D6390C6D5E8ED174901,22mar2016,2,1,0,2016,4,1,3,...,0.146,6.4,6.00,2,1,0,1,NaN,NaN,NaN
117102,0xFFAAD96EE841D318280B7616BD3B06CB,0x9B20928E594EEAF2720A067394A9912B,14aug2014,2,1,0,2014,1,3,8,...,0.039,5.3,5.70,1,0,0,0,NaN,NaN,NaN
117103,0xFFAAD96EE841D318280B7616BD3B06CB,0x3242DAC719090336CB37BF90D3B09456,27jun2016,2,2,1,2016,1,2,6,...,0.039,5.7,7.50,1,0,0,1,2.0,683.0,2.0
117147,0xFFD0C2E38B14DF2845BC012F8D52F442,0xC036D91963D26720D57CED9EC04171A3,04nov2016,2,2,1,2016,3,4,11,...,0.086,6.0,5.40,1,1,0,0,2.0,801.0,2.0
